In [3]:
%run "get_packages.py"

In [4]:
today = date.today()
end_date = today.strftime("%Y%m%d")
strt_date="20221230"
std_size = 0.03
concentration_size = 0.05
step_size = 0.005
TE_upper_bound = 0.012
TE_lower_bound = 0.008
features_weight = 0.33
multiplier = 2
carry_cost_lim = 0.003
start_date = 1/1/2003

REER_lag = 22
TB_lag = 30
GDP_lag = 120
CPI_lag = 30

Adj_REER = ["REER", "Conditional REER", "Symmetrical REER"]
Carry = ["Nominal Carry", "Real Carry"]
Conviction = ["Awa", "Agn"]

currencies = ["BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN", "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD", "KRW"]


In [3]:
#SPOT PRICE

bquery = blp.BlpQuery().start()
spot_query = bquery.bdh(
        ["BRL BGN Curncy", "CLP BGN Curncy", "CNY CMPN Curncy", "COP REGN Curncy", "CZK BGN Curncy", "HUF BGN Curncy", "IDR CMPN Curncy", "MYR CMPN Curncy", "MXN BGN Curncy", "PEN BGN Curncy", "PHP CMPN Curncy", "PLN BGN Curncy", "RON BGN Curncy", "RUB BGN Curncy", "ZAR BGN Curncy", "THB BGN Curncy", "TRY BGN Curncy", "INR CMPN Curncy", "SGD BGN Curncy", "TWD BGN Curncy", "KRW CMPN Curncy"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})

spot_query['security_short'] = spot_query['security'].str[:3]

spot = spot_query.pivot_table(index='date', columns='security_short', values='PX_LAST', aggfunc='first')
spot.reset_index( inplace=True)
spot.rename(columns={'date':'DATE'}, inplace=True)
spot['DATE'] = pd.to_datetime(spot['DATE'])
spot.sort_values(by='DATE', ascending=False, inplace=True)
spot.dropna(inplace=True)

# Extract relevant columns
columns = ["DATE"] + currencies
spot_subset = spot[columns]

# Calculate daily returns
spot_subset.set_index("DATE", inplace=True)
daily_return = spot_subset.shift(-1)/spot_subset  -1
daily_return.reset_index(inplace=True)
#daily_return.dropna(inplace=True)
spot_return = daily_return.melt(id_vars=["DATE"], var_name="Currency", value_name="Daily_Return")
spot_return

,DATE,Currency,Daily_Return
0,2024-10-15,BRL,-0.010437
1,2024-10-14,BRL,0.003111
2,2024-10-11,BRL,-0.004954
3,2024-10-10,BRL,0.001737
4,2024-10-09,BRL,-0.010495
...,...,...,...
9823,2023-01-05,KRW,-0.002094
9824,2023-01-04,KRW,0.005439
9825,2023-01-03,KRW,-0.007293
9826,2023-01-02,KRW,-0.007536


In [4]:
bquery = blp.BlpQuery().start()
MA_query = bquery.bdh(
        ["BRL Curncy","CLP Curncy","CNY Curncy","COP Curncy","CZK Curncy","HUF Curncy","IDR Curncy","MYR Curncy","MXN Curncy","PEN Curncy","PHP Curncy","PLN Curncy","RON Curncy","RUB Curncy","ZAR Curncy","THB Curncy","TRY Curncy","INR Curncy","SGD Curncy","TWD Curncy","KRW Curncy"],
        ["MOV_AVG_5D","MOV_AVG_20D"],
        start_date=strt_date,
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})
MA_query.rename(columns={'date':'DATE'}, inplace=True)
MA_query['Currency'] = MA_query['security'].str[:3]
MA_query.drop(columns="security", inplace=True)
MA_query['Momentum'] = np.where(MA_query['MOV_AVG_5D'] > MA_query['MOV_AVG_20D'], 1, -1)
Momentum = MA_query[["DATE", "Currency", "MOV_AVG_5D", "MOV_AVG_20D","Momentum"]]
Momentum['DATE'] = pd.to_datetime(Momentum['DATE'])
Momentum.sort_values(by='DATE', ascending=False, inplace=True)
Momentum.reset_index(drop=True, inplace=True)
#Momentum.dropna(inplace=True)
Momentum

C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)


,DATE,Currency,MOV_AVG_5D,MOV_AVG_20D,Momentum
0,2024-10-15,MYR,4.2939,4.2177,1
1,2024-10-15,INR,84.0215,83.8349,1
2,2024-10-15,TWD,32.1980,31.9970,1
3,2024-10-15,THB,33.4420,33.0270,1
4,2024-10-15,TRY,34.2398,34.1828,1
...,...,...,...,...,...
9823,2022-12-30,COP,4774.4300,4788.9700,-1
9824,2022-12-30,TWD,30.7160,30.6780,1
9825,2022-12-30,SGD,1.3457,1.3519,-1
9826,2022-12-30,RUB,NaN,NaN,-1


In [5]:
#3M Implied vol
bquery = blp.BlpQuery().start()
IMPVOL_query = bquery.bdh(
        ["USDBRLV3M Curncy","USDCLPV3M Curncy","USDCNYV3M Curncy","USDCOPV3M Curncy","USDCZKV3M Curncy","USDHUFV3M Curncy","USDIDRV3M Curncy","USDMYRV3M Curncy","USDMXNV3M Curncy","USDPENV3M Curncy","USDPHPV3M Curncy","USDPLNV3M Curncy","USDRONV3M Curncy","USDRUBV3M Curncy","USDZARV3M Curncy","USDTHBV3M Curncy","USDTRYV3M Curncy","USDINRV3M Curncy","USDSGDV3M Curncy","USDTWDV3M Curncy","USDKRWV3M Curncy"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"}) #

IMPVOL_query.rename(columns={'date':'DATE', 'PX_LAST':'3M Implied Vol'}, inplace=True)

conditions = [
    (IMPVOL_query['security'] == 'USDBRLV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCLPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCNYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCOPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCZKV3M Curncy'),
    (IMPVOL_query['security'] == 'USDHUFV3M Curncy'),
    (IMPVOL_query['security'] == 'USDIDRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMYRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMXNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPENV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPHPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPLNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRONV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRUBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDZARV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTHBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTRYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDINRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDSGDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTWDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDKRWV3M Curncy'),
]

IMPVOL_query['Currency'] = np.select(conditions, currencies, default=None)
IMPVOL_query.drop(columns="security", inplace=True)
IMPVOL_query.sort_values(by='DATE', ascending=False, inplace=True)
IMPVOL_query=IMPVOL_query[['DATE','Currency','3M Implied Vol']]
IMPVOL_query

,DATE,Currency,3M Implied Vol
9827,2024-10-15,MYR,7.0375
9817,2024-10-15,INR,2.6150
9807,2024-10-15,TWD,8.2700
9808,2024-10-15,THB,9.5500
9809,2024-10-15,TRY,14.7000
...,...,...,...
17,2022-12-30,COP,18.9750
18,2022-12-30,TWD,6.8150
19,2022-12-30,SGD,5.9400
20,2022-12-30,RUB,36.5450


In [25]:
import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np

all_currencies = ["BRL","CLP","CNY","COP","CZK","HUF","IDR","MYR","MXN","PEN","PHP","PLN","RON","RUB","ZAR","THB","TRY","INR","SGD","TWD","KRW","UAH","EGP","EUR","JPY","USD","GBP","CAD","ARS","CHF"]

# Create an empty list to store the dataframes
all_cpi_data = []

# Loop through the dates from 2020 to today
start_date = pd.Timestamp('2022-01-01')
end_date = pd.Timestamp('today') 
for date in pd.date_range(start=start_date, end=end_date, freq='MS'): # 'MS' for month start frequency
    specific_date = date + pd.DateOffset(months=1) - pd.DateOffset(days=1)  # Get the last day of the month

    begin = ((specific_date - pd.DateOffset(months=2)).replace(day=1) + pd.DateOffset(months=1) - pd.DateOffset(days=1) - BDay(1)).strftime('%Y%m%d')
    end = ((specific_date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=1) - pd.DateOffset(days=1) - BDay(1)).strftime('%Y%m%d')

    bquery = blp.BlpQuery().start()
    CPI_query = bquery.bdh(
            ["BZPIIPCM Index","CLINNSMO Index","CNCPIMOM Index","COCPIMOM Index","CZCPMOM Index","HUCPIMM Index","IDCPIM Index","MACPIMOM Index","MXCPCHNG Index","PRCPNONC Index","PHC2MOM Index","POCPIMOM Index","ROCOPMOM Index","RUCPIMOM Index","SACPIMOM Index","THCPIMOM Index","TUCPIM Index","INFUTOT Index","SICPIMOM Index","TWCPIMOM Index","KOCPIMOM Index","UACPTMOM Index","EGCPMOM Index","ECCPEMUM Index","JNCPIMOM Index","CPURNSA% Index","UKRPCJMR Index","CACPICHG Index","ARNCNINX Index","SZCPIMOM Index"],
            ["PX_LAST"],
            start_date= begin,
            end_date= end, 
            options={"adjustmentSplit": True, "nonTradingDayFillOption":"ALL_CALENDAR_DAYS", "periodicitySelection":"MONTHLY", 'nonTradingDayFillMethod':'NIL_VALUE'})
    CPI_query.rename(columns={'PX_LAST':'CPI'}, inplace=True)

    conditions = [
    (CPI_query['security'] == 'BZPIIPCM Index'),
    (CPI_query['security'] == 'CLINNSMO Index'),
    (CPI_query['security'] == 'CNCPIMOM Index'),
    (CPI_query['security'] == 'COCPIMOM Index'),
    (CPI_query['security'] == 'CZCPMOM Index'),
    (CPI_query['security'] == 'HUCPIMM Index'),
    (CPI_query['security'] == 'IDCPIM Index'),
    (CPI_query['security'] == 'MACPIMOM Index'),
    (CPI_query['security'] == 'MXCPCHNG Index'),
    (CPI_query['security'] == 'PRCPNONC Index'),
    (CPI_query['security'] == 'PHC2MOM Index'),
    (CPI_query['security'] == 'POCPIMOM Index'),
    (CPI_query['security'] == 'ROCOPMOM Index'),
    (CPI_query['security'] == 'RUCPIMOM Index'),
    (CPI_query['security'] == 'SACPIMOM Index'),
    (CPI_query['security'] == 'THCPIMOM Index'),
    (CPI_query['security'] == 'TUCPIM Index'),
    (CPI_query['security'] == 'INFUTOT Index'),
    (CPI_query['security'] == 'SICPIMOM Index'),
    (CPI_query['security'] == 'TWCPIMOM Index'),
    (CPI_query['security'] == 'KOCPIMOM Index'),
    (CPI_query['security'] == 'UACPTMOM Index'),
    (CPI_query['security'] == 'EGCPMOM Index'),
    (CPI_query['security'] == 'ECCPEMUM Index'),
    (CPI_query['security'] == 'JNCPIMOM Index'),
    (CPI_query['security'] == 'CPURNSA% Index'),
    (CPI_query['security'] == 'UKRPCJMR Index'),
    (CPI_query['security'] == 'CACPICHG Index'),
    (CPI_query['security'] == 'ARNCNINX Index'),
    (CPI_query['security'] == 'SZCPIMOM Index')
    ]

    CPI_query['Currency'] = np.select(conditions, all_currencies, default=None)
    CPI_query.loc[CPI_query['Currency'] == 'INR', 'CPI'] = 100*(CPI_query['CPI']/185.8 -1)
        
    CPI_query=CPI_query[['Currency','CPI']]
    CPI_query['Date'] = specific_date # Add a Date column

    all_cpi_data.append(CPI_query) # Append the dataframe to the list


# Concatenate all dataframes vertically
final_cpi_data = pd.concat(all_cpi_data, ignore_index=True)

print(final_cpi_data)

     Currency       CPI       Date
0         TRY  3.510000 2022-01-31
1         RUB  0.960000 2022-01-31
2         RON  0.000000 2022-01-31
3         PLN  1.000000 2022-01-31
4         PHP  0.600000 2022-01-31
...       ...       ...        ...
1015      SGD  0.700000 2024-10-31
1016      INR  3.875135 2024-10-31
1017      TRY  2.470000 2024-10-31
1018      IDR -0.030000 2024-10-31
1019      MYR  0.100000 2024-10-31

[1020 rows x 3 columns]


In [26]:
from pandas.tseries.offsets import MonthEnd

# Calculate the start and end dates for the entire period (month-end frequency)
start_date = final_cpi_data['Date'].min()
end_date = final_cpi_data['Date'].max()

# Create a daily date range for the entire period
daily_dates = pd.date_range(start=start_date - pd.DateOffset(days=1), end=end_date, freq='D')

# Create an empty dataframe for the daily CPI data
final_daily_cpi_data = pd.DataFrame(columns=['Date', 'Currency', 'CPI'])

# Loop through each month-end date in the CPI_query
for month_end_date in final_cpi_data['Date'].unique():
    # Filter the CPI_query for the current month-end
    month_data = final_cpi_data[final_cpi_data['Date'] == month_end_date]

    # Get the start and end dates for the current month
    start_of_month = month_end_date.replace(day=1)

    # Filter the daily_dates for the current month
    current_month_dates = daily_dates[(daily_dates >= start_of_month) & (daily_dates <= month_end_date)]

    # Repeat the month's CPI data for each day in the month (using cross-join)
    repeated_data = pd.merge(pd.DataFrame({'Date': current_month_dates}), month_data, how='cross')

    # Append the repeated data to the final dataframe
    final_daily_cpi_data = pd.concat([final_daily_cpi_data, repeated_data], ignore_index=True)

# Sort the final dataframe by date
final_daily_cpi_data = final_daily_cpi_data.sort_values('Date').reset_index(drop=True)

print(final_daily_cpi_data)


C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_32256/2254841379.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_daily_cpi_data = pd.concat([final_daily_cpi_data, repeated_data], ignore_index=True)


      Date Currency       CPI     Date_x     Date_y
0      NaN      TRY  3.510000 2022-01-30 2022-01-31
1      NaN      RUB  0.960000 2022-01-30 2022-01-31
2      NaN      RON  0.000000 2022-01-30 2022-01-31
3      NaN      PLN  1.000000 2022-01-30 2022-01-31
4      NaN      PHP  0.600000 2022-01-30 2022-01-31
...    ...      ...       ...        ...        ...
30175  NaN      SGD  0.700000 2024-10-31 2024-10-31
30176  NaN      INR  3.875135 2024-10-31 2024-10-31
30177  NaN      TRY  2.470000 2024-10-31 2024-10-31
30178  NaN      IDR -0.030000 2024-10-31 2024-10-31
30179  NaN      MYR  0.100000 2024-10-31 2024-10-31

[30180 rows x 5 columns]


In [5]:
start_date = pd.Timestamp('2023-11-20') 
end_date = pd.Timestamp('2024-01-01') 

# Create an empty list to store the daily dataframes
all_reer_data = []

# Loop through the dates from 2022 to today (using daily frequency)
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    bquery = blp.BlpQuery().start()

    REER_query = bquery.bdp(
            ["BISBBRR Index","BISBCLR Index","BISBCNR Index","BISBCOP Index","BISBCZR Index","BISBHUR Index","BISBIDR Index","BISBMYR Index","BISBMXR Index","BISBPER Index","BISBPHR Index","BISBPLR Index","BISBROR Index","BISBRUR Index","BISBZAR Index","BISBTHR Index","BISBTRR Index","BISBINR Index","BISBSGR Index","BISBTWR Index","BISBKRR Index"],
            ["PX_LAST", "LAST_UPDATE_DT"]  # Retrieve LAST_UPDATE_DT directly
            )
    REER_query.rename(columns={'PX_LAST':'REER'}, inplace=True)

    conditions = [

    (REER_query['security'] == 'BISBBRR Index'),
    (REER_query['security'] == 'BISBCLR Index'),
    (REER_query['security'] == 'BISBCNR Index'),
    (REER_query['security'] == 'BISBCOP Index'),
    (REER_query['security'] == 'BISBCZR Index'),
    (REER_query['security'] == 'BISBHUR Index'),
    (REER_query['security'] == 'BISBIDR Index'),
    (REER_query['security'] == 'BISBMYR Index'),
    (REER_query['security'] == 'BISBMXR Index'),
    (REER_query['security'] == 'BISBPER Index'),
    (REER_query['security'] == 'BISBPHR Index'),
    (REER_query['security'] == 'BISBPLR Index'),
    (REER_query['security'] == 'BISBROR Index'),
    (REER_query['security'] == 'BISBRUR Index'),
    (REER_query['security'] == 'BISBZAR Index'),
    (REER_query['security'] == 'BISBTHR Index'),
    (REER_query['security'] == 'BISBTRR Index'),
    (REER_query['security'] == 'BISBINR Index'),
    (REER_query['security'] == 'BISBSGR Index'),
    (REER_query['security'] == 'BISBTWR Index'),
    (REER_query['security'] == 'BISBKRR Index')    ]

    REER_query['Currency'] = np.select(conditions, currencies, default=None) 

    def get_5Y_average(row): # Modified function to work on rows
        security = row['security']
        LAST_UPDATE = row['LAST_UPDATE_DT']
        start_date = (LAST_UPDATE - relativedelta(months=60)).strftime('%Y-%m-%d')
        end_date = LAST_UPDATE.strftime('%Y-%m-%d')
        query_text = f"get(avg(px_last(dates=range({start_date}, {end_date}), Frq=M, fill=PREV))) for(['{security}'])"
        BQL_query = bquery.bql(expression=f"{query_text}")
        try:
            avg_value = BQL_query.loc[BQL_query['security'] == security, 'value'].iloc[0]
        except IndexError:
            avg_value = None
        return avg_value

    REER_query['REER_5Y_avg'] = REER_query.apply(get_5Y_average, axis=1)
    REER_query.reset_index(inplace=True)
    REER_query=REER_query[['Currency', 'REER', 'REER_5Y_avg']]
    REER_query['Date'] = date  # Add the date column

    all_reer_data.append(REER_query)

# Concatenate all daily dataframes vertically
final_reer_data = pd.concat(all_reer_data, ignore_index=True)

print(final_reer_data)

    Currency    REER REER_5Y_avg       Date
0        KRW   94.29        None 2023-11-20
1        BRL  103.74        None 2023-11-20
2        CLP  101.65        None 2023-11-20
3        CNY   91.29        None 2023-11-20
4        COP  109.20        None 2023-11-20
..       ...     ...         ...        ...
898      THB   94.44        None 2024-01-01
899      TRY   94.94        None 2024-01-01
900      INR  101.43        None 2024-01-01
901      SGD  116.18        None 2024-01-01
902      TWD   96.75        None 2024-01-01

[903 rows x 4 columns]


In [6]:
final_reer_data

NameError: name 'final_reer_data' is not defined

In [6]:
#REER discount 
bquery = blp.BlpQuery().start()

REER_query = bquery.bdp(
        ["BISBBRR Index","BISBCLR Index","BISBCNR Index","BISBCOP Index","BISBCZR Index","BISBHUR Index","BISBIDR Index","BISBMYR Index","BISBMXR Index","BISBPER Index","BISBPHR Index","BISBPLR Index","BISBROR Index","BISBRUR Index","BISBZAR Index","BISBTHR Index","BISBTRR Index","BISBINR Index","BISBSGR Index","BISBTWR Index","BISBKRR Index"],
        ["PX_LAST"])
REER_query.rename(columns={'PX_LAST':'REER'}, inplace=True)

conditions = [
    (REER_query['security'] == 'BISBBRR Index'),
    (REER_query['security'] == 'BISBCLR Index'),
    (REER_query['security'] == 'BISBCNR Index'),
    (REER_query['security'] == 'BISBCOP Index'),
    (REER_query['security'] == 'BISBCZR Index'),
    (REER_query['security'] == 'BISBHUR Index'),
    (REER_query['security'] == 'BISBIDR Index'),
    (REER_query['security'] == 'BISBMYR Index'),
    (REER_query['security'] == 'BISBMXR Index'),
    (REER_query['security'] == 'BISBPER Index'),
    (REER_query['security'] == 'BISBPHR Index'),
    (REER_query['security'] == 'BISBPLR Index'),
    (REER_query['security'] == 'BISBROR Index'),
    (REER_query['security'] == 'BISBRUR Index'),
    (REER_query['security'] == 'BISBZAR Index'),
    (REER_query['security'] == 'BISBTHR Index'),
    (REER_query['security'] == 'BISBTRR Index'),
    (REER_query['security'] == 'BISBINR Index'),
    (REER_query['security'] == 'BISBSGR Index'),
    (REER_query['security'] == 'BISBTWR Index'),
    (REER_query['security'] == 'BISBKRR Index')
]

REER_query['Currency'] = np.select(conditions, currencies, default=None)

def get_5Y_average(df):
    security = df['security']
    LAST_UPDATE = bquery.bdp( [f"{security}"],["LAST_UPDATE_DT"]).loc[0,'LAST_UPDATE_DT']
    start_date = (LAST_UPDATE - relativedelta(months=60)).strftime('%Y-%m-%d')
    end_date = LAST_UPDATE.strftime('%Y-%m-%d')
    query_text = f"get(avg(px_last(dates=range({start_date}, {end_date}), Frq=M, fill=PREV))) for(['{security}'])"
    BQL_query = bquery.bql(expression=f"{query_text}")
    return BQL_query.loc[1,'value']

REER_query['REER_5Y_avg'] = REER_query.apply(get_5Y_average, axis=1)
REER_query.reset_index(inplace=True)
REER_query=REER_query[['Currency', 'REER', 'REER_5Y_avg']]

REER_query


KeyError: 1